# Incorporate marine species to NRC
### In this notebook we incorporate marine data to the updated terrestrial data calculated in the Terrestrial_NRC notebook
April 2022

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import arcgis
from arcgis.gis import GIS
import json
import pandas as pd
from arcgis.features import FeatureLayerCollection
import requests as re
from copy import deepcopy
from itertools import repeat
import functools

## Import, explore and prepare the data

In [2]:
path = '/Users/sofia/Documents/HE_Data/NRC/NRC_Marine'

In [3]:
# Import tables
gadm = pd.read_csv('/Users/sofia/Documents/HE_Data/NRC/NRC_Terrestrial/NRC_Terrestrial_20220412.csv') # This is the updated gadm_centroid
spi = pd.read_csv(f'{path}/Marine_SPI_by_country_202203323_ter-1.csv')# table with spi and protection
mar = pd.read_csv(f'{path}/NRC_marine_species_data_20220323_ter-1.csv') # Species able provided by Alex (MOL)
pop = pd.read_csv(f'{path}/Pop2020_EEZ.csv') # calculated in arcgis pro using the population2020.crf and the EEZ shapefile
hm = pd.read_csv(f'{path}/marine_perc_human.csv') # human modification table
EEZ = pd.read_csv(f'{path}/eez_dissolve_centroid.csv') # centroids of the EEZ created in ArcGIS pro (inside option deactivated so the centroid can fall outside the geometry)

In [ ]:
gadm.head(1)

,Unnamed: 0,GID_0,NAME_0,jpg_url,GNI_PPP,max_amph,max_bird,max_mamm,max_rept,max_cact,...,birds,mammals,reptiles,endemic_amphibians,endemic_birds,endemic_mammals,endemic_reptiles,nspecies_ter,total_endemic_ter,filter_similar_ter
0,0,ABW,Aruba,https://live.staticflickr.com/1952/31416683438...,3.9,3,1,10,10,10,...,193,4,32,0,0,0,3,232,3,"{""filter_Area_Country"": [""JEY"", ""CXR"", ""WLF"", ..."


In [5]:
len(gadm)

252

In [6]:
spi.head(1)

,SOVEREIGN1,iso_ter1,year,nspecies,SPI_low,SPI_high,percentprotected_low,percentprotected_high
0,Albania,ALB,1980,347,0.0,0.0,0.0,0.0


In [7]:
len(spi['iso_ter1'].unique()) # There are 183 unique countries/territories with SPI data

183

In [8]:
mar.head(1)

,speciesgroup,species,countryname,iso_ter1,percentprotected,NSPS,stewardship
0,marine fishes,Abalistes filamentosus,Australia,AUS,25-50%,75-100,22


In [9]:
len(mar['iso_ter1'].unique()) # There are 183 unique countries/territories with marine species

183

In [10]:
pop.head(1)

,OID_,GID_0,ZONE_CODE,COUNT,AREA,SUM
0,1,ABW,1,55.0,0.003819,1547.605201


In [11]:
len(pop['GID_0'].unique()) # There are 200 countries/territories that have population in their EEZ

200

In [12]:
hm.head(1)

,Unnamed: 0,ISO_TER1_F,no_human,human,very_high
0,2,ABW,0.0,0.783047,0.216953


In [13]:
len(hm['ISO_TER1_F'].unique()) # There are 203 countries/territories that have human modification data in their EEZ

203

In [14]:
EEZ.head()

,OID_,GID_0,AREA_KM2,ORIG_FID,Latitude,Longitude
0,1,ABW,29970.299588,1,13.741380,-69.673412
1,2,AGO,495859.762742,2,-11.701098,11.035371
2,3,AIA,90157.964205,3,20.018030,-62.543285
3,4,ALB,12165.548773,4,40.927084,19.113001
4,5,ARE,57838.146798,5,25.058911,54.030580


In [15]:
len(EEZ['GID_0'].unique()) # There are 201 countries/territories that have EEZ

201

#### Check the GID_0 to see if there are discrepancies with gadm

In [4]:
# Number of countries in species list according to GID_0
country_list = list(mar['iso_ter1'].unique())
len(country_list) ## 183 territories in the species table

183

In [5]:
# Find GID_0 that are in species table but not in gadm 
list1= list(gadm['GID_0'])
list2=list(mar['iso_ter1'].unique())
list(set(list2).difference(list1))  # XXZ is international waters and UMI is the United States Minor Outlying Islands. None of them are in gadm so let's remove them

['UMI', 'XXZ']

In [6]:
# Remove 'XXZ', 'UMI' from species table to have only GID_0 that are in gadm
l = list(set(list2).difference(list1))
mar = mar[~mar['iso_ter1'].isin(l)]
len(mar.iso_ter1.unique())

181

In [7]:
# Find GID_0 that are in spi table but not in gadm:
list1= list(gadm['GID_0'])
list2=list(spi['iso_ter1'].unique())
list(set(list2).difference(list1)) 

['UMI', 'XXZ']

In [8]:
# Remove 'XXZ', 'UMI' from species table to have only GID_0 that are in gadm
l = list(set(list2).difference(list1))
spi = spi[~spi['iso_ter1'].isin(l)]
len(mar.iso_ter1.unique())

181

In [9]:
# Find GID_0 that are in hm but not in gadm:
list1= list(gadm['GID_0'])
list2=list(hm['ISO_TER1_F'].unique())
list(set(list2).difference(list1)) 

['UMI', 'TMP']

In [10]:
# Remove 'XXZ', 'UMI' from hm table to have only GID_0 that are in gadm
l = list(set(list2).difference(list1))
hm = hm[~hm['ISO_TER1_F'].isin(l)]
len(hm.ISO_TER1_F.unique())

201

In [11]:
# Find GID_0 that are in EEZ table but not in gadm
list1= list(gadm['GID_0'])
list2=list(EEZ['GID_0'].unique())
list(set(list2).difference(list1)) 

[]

In [12]:
# Find GID_0 that are in population but not in gadm:
list1= list(gadm['GID_0'])
list2=list(pop['GID_0'].unique())
list(set(list2).difference(list1)) 

[]

#### Note: Although there are 201 EEZ (and population and human modification for those), the species and the spi datasets only have info for 181 countries. So we'll only consider those for the marine NRC. Let's remove the GID_0 that are not present in the species table

In [13]:
# Find mismatches in GID_0 between mar table and hm table:
list1= list(mar['iso_ter1'])
list2=list(hm['ISO_TER1_F'].unique())
list3=list(set(list2).difference(list1))
# Keep only records which GID_0 is in the species dataset (mar)
hm2 = hm[~hm['ISO_TER1_F'].isin(list3)]
len(hm2)

181

In [14]:
list1= list(mar['iso_ter1'])
list2=list(hm2['ISO_TER1_F'].unique())
list(set(list2).difference(list1)) # Now all hm have species data

[]

In [15]:
# Change name of human modification fields to prepare it for join
hm2 = hm2.rename(columns={'ISO_TER1_F':'GID_0', 'no_human':'hm_no_mar', 'human':'hm_mar', 'very_high':'hm_vh_mar'}).drop(columns= {'Unnamed: 0'})
hm2.head()

,GID_0,hm_no_mar,hm_mar,hm_vh_mar
0,ABW,0.000000,0.783047,0.216953
2,AIA,0.000000,0.943677,0.056323
3,ALB,0.000263,0.003746,0.995991
4,ARE,0.000116,0.034045,0.965838
5,ARG,0.000635,0.906946,0.092419


In [16]:
# The human modification table was given in 0-1 values, let's make it in % to match the terrestrial data
hm2['hm_no_mar'] = hm2['hm_no_mar'].apply(lambda x: x*100)
hm2['hm_mar'] = hm2['hm_mar'].apply(lambda x: x*100)
hm2['hm_vh_mar'] = hm2['hm_vh_mar'].apply(lambda x: x*100)
hm2.head()

,GID_0,hm_no_mar,hm_mar,hm_vh_mar
0,ABW,0.000000,78.304750,21.695250
2,AIA,0.000000,94.367741,5.632259
3,ALB,0.026286,0.374581,99.599133
4,ARE,0.011637,3.404538,96.583825
5,ARG,0.063502,90.694610,9.241887


In [17]:
# Find mismatches in GID_0 between mar table and population table:
list1= list(mar['iso_ter1'])
list2=list(pop['GID_0'].unique())
list3=list(set(list2).difference(list1)) 
# Keep only records which GID_0 is in the species dataset (mar)
pop2 = pop[~pop['GID_0'].isin(list3)]
len(pop2)

180

In [18]:
list1= list(mar['iso_ter1'])
list2=list(pop2['GID_0'].unique())
list(set(list1).difference(list2)) # ATA is the only GID_0 in the species dataset that is not included in pop table (pop=0)

['ATA']

In [19]:
# Change name pop fields to prepare it for join
pop2 = pop2.drop(columns={'OID_', 'ZONE_CODE', 'COUNT', 'AREA'}).rename(columns = {'SUM':'Pop2020_EEZ'})
pop2.head(2)

,GID_0,Pop2020_EEZ
0,ABW,1547.605201
2,AIA,1693.719824


In [20]:
# Find mismatches in GID_0 between mar table and EEZ table:
list1= list(mar['iso_ter1'])
list2=list(EEZ['GID_0'].unique())
list3=list(set(list2).difference(list1))
# Keep only records which GID_0 is in the species dataset (mar)
eez2 = EEZ[~EEZ['GID_0'].isin(list3)]
len(eez2)

181

------------------------------------------------------------------------------------------------------------------
## Overview tab: generate general information
### Add Population data 
Population by EEZ was calculated in ArcGIS Pro using the EEZ_dissolved (by GID_0) shp and the population2020.crf

In [71]:
gadm.columns

Index(['Unnamed: 0', 'GID_0', 'NAME_0', 'jpg_url', 'GNI_PPP', 'max_amph',
       'max_bird', 'max_mamm', 'max_rept', 'max_cact', 'max_coni', 'max_all',
       'sentence', 'Average SPI', 'GlobalID', 'continent', 'has_priority',
       'has_raisg', 'Area_Country', 'protection_needed_ter', 'iso2',
       'max_highlited_sp', 'x', 'y', 'hm_ter', 'hm_no_ter', 'hm_vh_ter',
       'Pop2020', 'SPI_ter', 'prop_protected_ter', 'amphibians', 'birds',
       'mammals', 'reptiles', 'endemic_amphibians', 'endemic_birds',
       'endemic_mammals', 'endemic_reptiles', 'nspecies_ter',
       'total_endemic_ter', 'filter_similar_ter'],
      dtype='object')

In [79]:
len(pop2)

180

In [23]:
# Add new population data and remove old field (SUM)
df = pd.merge(gadm, pop2,  how='left', left_on=['GID_0'], right_on = ['GID_0']).drop(columns={'Unnamed: 0'})
df.columns

Index(['GID_0', 'NAME_0', 'jpg_url', 'GNI_PPP', 'max_amph', 'max_bird',
       'max_mamm', 'max_rept', 'max_cact', 'max_coni', 'max_all', 'sentence',
       'Average SPI', 'GlobalID', 'continent', 'has_priority', 'has_raisg',
       'Area_Country', 'protection_needed_ter', 'iso2', 'max_highlited_sp',
       'x', 'y', 'hm_ter', 'hm_no_ter', 'hm_vh_ter', 'Pop2020', 'SPI_ter',
       'prop_protected_ter', 'amphibians', 'birds', 'mammals', 'reptiles',
       'endemic_amphibians', 'endemic_birds', 'endemic_mammals',
       'endemic_reptiles', 'nspecies_ter', 'total_endemic_ter',
       'filter_similar_ter', 'Pop2020_EEZ'],
      dtype='object')

In [24]:
df[df.GID_0=='ATA']

,GID_0,NAME_0,jpg_url,GNI_PPP,max_amph,max_bird,max_mamm,max_rept,max_cact,max_coni,...,mammals,reptiles,endemic_amphibians,endemic_birds,endemic_mammals,endemic_reptiles,nspecies_ter,total_endemic_ter,filter_similar_ter,Pop2020_EEZ
11,ATA,Antarctica,https://live.staticflickr.com/1590/25126847203...,NaN,0,10,10,0,0,0,...,0,0,0,0,0,0,97,0,"{""filter_Area_Country"": [""COD"", ""KAZ"", ""ARG"", ...",NaN


### SPI and % Protected
MOL sent a first version with these values, which is in https://eowilson.maps.arcgis.com/home/item.html?id=d48f5ea1e59a42048f57e4c44c1a82a3. However, in this case they grouped the EEZ by sovereign1, so they didn't match the terrestrial NRC, which are divided by territories (French Guiana is shown independently from France). So MOL sent a new table dividing the EEZ by iso_ter1 (when this was null they used the field iso_sov1). This was done for both tables (SPI&Protection and species)

In [25]:
# Take the last value (2021) for the general overview
last = spi[spi['year']==2021].copy()
last.head(5)

,SOVEREIGN1,iso_ter1,year,nspecies,SPI_low,SPI_high,percentprotected_low,percentprotected_high
41,Albania,ALB,2021,347,1.86,1.86,0.69,0.69
83,Algeria,DZA,2021,489,0.22,0.22,0.02,0.02
125,Antarctica,ATA,2021,121,50.79,50.79,35.55,35.55
167,Antigua and Barbuda,ATG,2021,1519,9.04,9.04,0.30,0.30
209,Argentina,ARG,2021,671,7.00,7.00,9.25,9.25


In [26]:
# Take only relevant fields and change their names (include marine in name to distinguish them from terrestrial)
last = last[['iso_ter1','SPI_high', 'percentprotected_high']]
last = last.rename(columns= {'SPI_high':'SPI_mar', 'percentprotected_high':'prop_protected_mar', 'iso_ter1':'GID_0'})
last.head(1)

,GID_0,SPI_mar,prop_protected_mar
41,ALB,1.86,0.69


In [27]:
len(last) # There are 181 territories with marine SPI and protection data

181

In [28]:
# Create new dataframe with the merge of the gadm_centroid and the new values
df= pd.merge(df, last ,how='left', left_on=['GID_0'], right_on = ['GID_0'])
df.head(1)

,GID_0,NAME_0,jpg_url,GNI_PPP,max_amph,max_bird,max_mamm,max_rept,max_cact,max_coni,...,endemic_amphibians,endemic_birds,endemic_mammals,endemic_reptiles,nspecies_ter,total_endemic_ter,filter_similar_ter,Pop2020_EEZ,SPI_mar,prop_protected_mar
0,ABW,Aruba,https://live.staticflickr.com/1952/31416683438...,3.9,3,1,10,10,10,0,...,0,0,0,3,232,3,"{""filter_Area_Country"": [""JEY"", ""CXR"", ""WLF"", ...",1547.605201,0.0,0.0


### Add human modification data

In [29]:
hm2.head()

,GID_0,hm_no_mar,hm_mar,hm_vh_mar
0,ABW,0.000000,78.304750,21.695250
2,AIA,0.000000,94.367741,5.632259
3,ALB,0.026286,0.374581,99.599133
4,ARE,0.011637,3.404538,96.583825
5,ARG,0.063502,90.694610,9.241887


In [30]:
len(hm2)

181

In [31]:
# Create new dataframe with the merge of the gadm_centroid and the new values
df= pd.merge(df, hm2 ,how='left', left_on=['GID_0'], right_on = ['GID_0'])
df.head(1)

,GID_0,NAME_0,jpg_url,GNI_PPP,max_amph,max_bird,max_mamm,max_rept,max_cact,max_coni,...,endemic_reptiles,nspecies_ter,total_endemic_ter,filter_similar_ter,Pop2020_EEZ,SPI_mar,prop_protected_mar,hm_no_mar,hm_mar,hm_vh_mar
0,ABW,Aruba,https://live.staticflickr.com/1952/31416683438...,3.9,3,1,10,10,10,0,...,3,232,3,"{""filter_Area_Country"": [""JEY"", ""CXR"", ""WLF"", ...",1547.605201,0.0,0.0,0.0,78.30475,21.69525


### Calculate number of marine species and endemic species

In [32]:
mar.head(1)

,speciesgroup,species,countryname,iso_ter1,percentprotected,NSPS,stewardship
0,marine fishes,Abalistes filamentosus,Australia,AUS,25-50%,75-100,22


In [33]:
## Get number of species (by taxa) per country
mar2 = mar[['speciesgroup', 'species','countryname', 'iso_ter1']]
mar_num = mar2.groupby(by = ['speciesgroup', 'countryname', 'iso_ter1']).count().reset_index()
mar_num.head(5)

,speciesgroup,countryname,iso_ter1,species
0,marine fishes,Albania,ALB,345
1,marine fishes,Algeria,DZA,470
2,marine fishes,Antarctica,ATA,106
3,marine fishes,Antigua and Barbuda,ATG,1493
4,marine fishes,Argentina,ARG,632


In [34]:
## Distinguish between taxa
mamm = mar_num[mar_num['speciesgroup']=='marine mammals']
fish = mar_num[mar_num['speciesgroup']=='marine fishes']

In [35]:
mamm.head(1)

,speciesgroup,countryname,iso_ter1,species
181,marine mammals,Albania,ALB,2


In [36]:
fish.head(1)

,speciesgroup,countryname,iso_ter1,species
0,marine fishes,Albania,ALB,345


In [37]:
## Include number of species in dataframe
df= pd.merge(df, mamm,  how='left', left_on=['GID_0'], right_on = ['iso_ter1']).drop(columns = {'speciesgroup','iso_ter1','countryname'}).rename(columns={'species': 'mammals_mar'})
df= pd.merge(df, fish,  how='left', left_on=['GID_0'], right_on = ['iso_ter1']).drop(columns = {'speciesgroup','iso_ter1','countryname'}).rename(columns={'species': 'fishes_mar'})
df.head(5)

,GID_0,NAME_0,jpg_url,GNI_PPP,max_amph,max_bird,max_mamm,max_rept,max_cact,max_coni,...,total_endemic_ter,filter_similar_ter,Pop2020_EEZ,SPI_mar,prop_protected_mar,hm_no_mar,hm_mar,hm_vh_mar,mammals_mar,fishes_mar
0,ABW,Aruba,https://live.staticflickr.com/1952/31416683438...,3.9,3,1,10,10,10,0,...,3,"{""filter_Area_Country"": [""JEY"", ""CXR"", ""WLF"", ...",1547.605201,0.00,0.00,0.0,78.304750,21.695250,19.0,1466.0
1,AFG,Afghanistan,https://p1.pxfuel.com/preview/967/12/53/afghan...,70.6,10,9,10,10,0,9,...,5,"{""filter_Area_Country"": [""MDG"", ""UKR"", ""CAF"", ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AGO,Angola,https://live.staticflickr.com/3787/13698381215...,192.2,10,10,10,10,3,5,...,66,"{""filter_Area_Country"": [""BOL"", ""ETH"", ""COL"", ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AIA,Anguilla,https://live.staticflickr.com/8063/8194570372_...,NaN,9,7,10,10,6,0,...,1,"{""filter_Area_Country"": [""PCN"", ""MAF"", ""SMR"", ...",1693.719824,4.68,0.28,0.0,94.367741,5.632259,25.0,1494.0
4,ALA,Åland,https://p1.pxfuel.com/preview/294/670/561/alan...,NaN,1,1,1,1,0,1,...,0,"{""filter_Area_Country"": [""STP"", ""KIR"", ""MTQ"", ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
## Calculate number of endemic species per country: mammals
mamm_e = mar[(mar['speciesgroup']=='marine mammals')&(mar['stewardship']==1)]
mamm_e = mamm_e.groupby(['speciesgroup','iso_ter1']).sum()
mamm_e = mamm_e.reset_index().rename(columns={'stewardship':'endemic_mammals_mar'})
mamm_e.head(5)

,speciesgroup,iso_ter1,endemic_mammals_mar
0,marine mammals,AUS,1
1,marine mammals,ECU,2
2,marine mammals,NZL,1
3,marine mammals,USA,1


In [39]:
## Calculate number of endemic species per country: fishes
fish_e = mar[(mar['speciesgroup']=='marine fishes')&(mar['stewardship']==1)]
fish_e = fish_e.groupby(['speciesgroup','iso_ter1']).sum()
fish_e = fish_e.reset_index().rename(columns={'stewardship':'endemic_fishes_mar'})
fish_e.head(5)

,speciesgroup,iso_ter1,endemic_fishes_mar
0,marine fishes,ARG,2
1,marine fishes,ATA,5
2,marine fishes,ATF,3
3,marine fishes,AUS,379
4,marine fishes,BHS,2


In [40]:
## Merge endemic data in dataframe
df= pd.merge(df, mamm_e,  how='left', left_on=['GID_0'], right_on = ['iso_ter1']).drop(columns = {'speciesgroup','iso_ter1'})
df= pd.merge(df, fish_e,  how='left', left_on=['GID_0'], right_on = ['iso_ter1']).drop(columns = {'speciesgroup','iso_ter1'})
df.head(5)

,GID_0,NAME_0,jpg_url,GNI_PPP,max_amph,max_bird,max_mamm,max_rept,max_cact,max_coni,...,Pop2020_EEZ,SPI_mar,prop_protected_mar,hm_no_mar,hm_mar,hm_vh_mar,mammals_mar,fishes_mar,endemic_mammals_mar,endemic_fishes_mar
0,ABW,Aruba,https://live.staticflickr.com/1952/31416683438...,3.9,3,1,10,10,10,0,...,1547.605201,0.00,0.00,0.0,78.304750,21.695250,19.0,1466.0,NaN,NaN
1,AFG,Afghanistan,https://p1.pxfuel.com/preview/967/12/53/afghan...,70.6,10,9,10,10,0,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AGO,Angola,https://live.staticflickr.com/3787/13698381215...,192.2,10,10,10,10,3,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AIA,Anguilla,https://live.staticflickr.com/8063/8194570372_...,NaN,9,7,10,10,6,0,...,1693.719824,4.68,0.28,0.0,94.367741,5.632259,25.0,1494.0,NaN,NaN
4,ALA,Åland,https://p1.pxfuel.com/preview/294/670/561/alan...,NaN,1,1,1,1,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
sum(df['endemic_fishes_mar'].notnull())

44

In [42]:
## Make columns type integer
cols = ['SPI_mar','mammals_mar', 'endemic_mammals_mar', 'fishes_mar', 'endemic_fishes_mar']
df[cols] = df[cols].fillna(0) 
df[cols] = df[cols].astype(int)

In [43]:
df.columns

Index(['GID_0', 'NAME_0', 'jpg_url', 'GNI_PPP', 'max_amph', 'max_bird',
       'max_mamm', 'max_rept', 'max_cact', 'max_coni', 'max_all', 'sentence',
       'Average SPI', 'GlobalID', 'continent', 'has_priority', 'has_raisg',
       'Area_Country', 'protection_needed_ter', 'iso2', 'max_highlited_sp',
       'x', 'y', 'hm_ter', 'hm_no_ter', 'hm_vh_ter', 'Pop2020', 'SPI_ter',
       'prop_protected_ter', 'amphibians', 'birds', 'mammals', 'reptiles',
       'endemic_amphibians', 'endemic_birds', 'endemic_mammals',
       'endemic_reptiles', 'nspecies_ter', 'total_endemic_ter',
       'filter_similar_ter', 'Pop2020_EEZ', 'SPI_mar', 'prop_protected_mar',
       'hm_no_mar', 'hm_mar', 'hm_vh_mar', 'mammals_mar', 'fishes_mar',
       'endemic_mammals_mar', 'endemic_fishes_mar'],
      dtype='object')

In [44]:
# Calculate total number of species and total number of endemic marine
df['nspecies_mar']= df['mammals_mar']+df['fishes_mar']
df['total_endemic_mar']= df['endemic_mammals_mar']+df['endemic_fishes_mar']
df.head(5)

,GID_0,NAME_0,jpg_url,GNI_PPP,max_amph,max_bird,max_mamm,max_rept,max_cact,max_coni,...,prop_protected_mar,hm_no_mar,hm_mar,hm_vh_mar,mammals_mar,fishes_mar,endemic_mammals_mar,endemic_fishes_mar,nspecies_mar,total_endemic_mar
0,ABW,Aruba,https://live.staticflickr.com/1952/31416683438...,3.9,3,1,10,10,10,0,...,0.00,0.0,78.304750,21.695250,19,1466,0,0,1485,0
1,AFG,Afghanistan,https://p1.pxfuel.com/preview/967/12/53/afghan...,70.6,10,9,10,10,0,9,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
2,AGO,Angola,https://live.staticflickr.com/3787/13698381215...,192.2,10,10,10,10,3,5,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
3,AIA,Anguilla,https://live.staticflickr.com/8063/8194570372_...,NaN,9,7,10,10,6,0,...,0.28,0.0,94.367741,5.632259,25,1494,0,0,1519,0
4,ALA,Åland,https://p1.pxfuel.com/preview/294/670/561/alan...,NaN,1,1,1,1,0,1,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0


In [45]:
# How many countries have marine data?
len(df[df['nspecies_mar']>0]) # 182

181

In [46]:
# How many countries do not have marine data?
len(df[df['nspecies_mar']==0]) # 71

71

---------------------------------------------------------------------------------------------------------------------------------------------
## Process to create the challenges tab (create array with similar filters)
### Create matrix to identify countries with shared stewardship to create the stewardship filter
This code is more efficient than that described in the notebook "shared_stewardship", which was used during the first iteration of the NRC

In [47]:
mar.head()

,speciesgroup,species,countryname,iso_ter1,percentprotected,NSPS,stewardship
0,marine fishes,Abalistes filamentosus,Australia,AUS,25-50%,75-100,22
1,marine fishes,Abalistes filamentosus,Australia,CCK,0-25%,0-25,22
2,marine fishes,Abalistes filamentosus,Australia,NFK,75-100%,75-100,22
3,marine fishes,Abalistes filamentosus,East Timor,TLS,0-25%,0-25,22
4,marine fishes,Abalistes filamentosus,Fiji,FJI,0-25%,50-75,22


In [48]:
# Create a copy with only the species name and the iso3
mar2 = mar[['iso_ter1','species']].copy()
mar2.head(5)

,iso_ter1,species
0,AUS,Abalistes filamentosus
1,CCK,Abalistes filamentosus
2,NFK,Abalistes filamentosus
3,TLS,Abalistes filamentosus
4,FJI,Abalistes filamentosus


In [49]:
%%time
# Create a matrix that has, for each country, the number of shared species with each of the other countries
m = mar2.merge(mar2, on='species') # perform a self-merge based on the species
mat = pd.crosstab(m.iso_ter1_x, m.iso_ter1_y) # perform crosstabulation operation
mat.reset_index(inplace=True)
mat= mat.rename(columns = {'iso_ter1_x':'index'})
mat.head(5)

CPU times: user 2.76 s, sys: 810 ms, total: 3.57 s
Wall time: 3.74 s


iso_ter1_y,index,ABW,AIA,ALB,ARE,ARG,ASM,ATA,ATF,ATG,...,USA,VCT,VEN,VGB,VIR,VNM,VUT,WSM,YEM,ZAF
0,ABW,1485,1327,61,133,187,257,3,329,1328,...,1435,1380,1485,1280,1329,300,325,279,116,441
1,AIA,1327,1519,58,118,204,300,4,339,1503,...,1462,1374,1492,1441,1498,273,352,319,111,460
2,ALB,61,58,347,31,38,24,0,49,58,...,78,59,61,54,57,36,28,27,23,99
3,ARE,133,118,31,1142,50,327,0,905,121,...,485,132,133,103,123,823,522,435,1023,816
4,ARG,187,204,38,50,671,136,32,308,203,...,393,217,248,182,205,93,175,162,33,386


In [50]:
mat.shape 

(181, 182)

In [51]:
# Save local copy
mat.to_csv('/Users/sofia/Documents/HE_Data/NRC/NRC_Marine/stewardship_matrix.csv',index=False)

### Get shared stewardship countries
Using the stewardship matrix. 

In [52]:
mat.columns.values

array(['index', 'ABW', 'AIA', 'ALB', 'ARE', 'ARG', 'ASM', 'ATA', 'ATF',
       'ATG', 'AUS', 'BEL', 'BES', 'BGD', 'BGR', 'BHS', 'BIH', 'BLM',
       'BLZ', 'BMU', 'BRA', 'BRB', 'BRN', 'CAN', 'CCK', 'CHL', 'CHN',
       'CMR', 'COD', 'COG', 'COK', 'COL', 'COM', 'CPV', 'CRI', 'CUB',
       'CUW', 'CYM', 'CYP', 'DEU', 'DMA', 'DNK', 'DOM', 'DZA', 'ECU',
       'EGY', 'ESH', 'ESP', 'EST', 'FIN', 'FJI', 'FLK', 'FRA', 'FRO',
       'FSM', 'GAB', 'GBR', 'GEO', 'GGY', 'GIN', 'GLP', 'GMB', 'GNB',
       'GNQ', 'GRC', 'GRD', 'GRL', 'GTM', 'GUF', 'GUM', 'GUY', 'HMD',
       'HND', 'HRV', 'HTI', 'IDN', 'IND', 'IRL', 'IRN', 'IRQ', 'ISL',
       'ISR', 'ITA', 'JAM', 'JEY', 'JOR', 'JPN', 'KEN', 'KHM', 'KIR',
       'KNA', 'KOR', 'KWT', 'LBN', 'LBR', 'LCA', 'LKA', 'LTU', 'LVA',
       'MAF', 'MAR', 'MCO', 'MDG', 'MDV', 'MEX', 'MHL', 'MLT', 'MMR',
       'MNP', 'MOZ', 'MRT', 'MSR', 'MTQ', 'MUS', 'MYS', 'MYT', 'NAM',
       'NCL', 'NFK', 'NGA', 'NIC', 'NIU', 'NLD', 'NOR', 'NZL', 'OMN',
       'PAK', 'PAN

In [53]:
mat.columns.values[1:183]

array(['ABW', 'AIA', 'ALB', 'ARE', 'ARG', 'ASM', 'ATA', 'ATF', 'ATG',
       'AUS', 'BEL', 'BES', 'BGD', 'BGR', 'BHS', 'BIH', 'BLM', 'BLZ',
       'BMU', 'BRA', 'BRB', 'BRN', 'CAN', 'CCK', 'CHL', 'CHN', 'CMR',
       'COD', 'COG', 'COK', 'COL', 'COM', 'CPV', 'CRI', 'CUB', 'CUW',
       'CYM', 'CYP', 'DEU', 'DMA', 'DNK', 'DOM', 'DZA', 'ECU', 'EGY',
       'ESH', 'ESP', 'EST', 'FIN', 'FJI', 'FLK', 'FRA', 'FRO', 'FSM',
       'GAB', 'GBR', 'GEO', 'GGY', 'GIN', 'GLP', 'GMB', 'GNB', 'GNQ',
       'GRC', 'GRD', 'GRL', 'GTM', 'GUF', 'GUM', 'GUY', 'HMD', 'HND',
       'HRV', 'HTI', 'IDN', 'IND', 'IRL', 'IRN', 'IRQ', 'ISL', 'ISR',
       'ITA', 'JAM', 'JEY', 'JOR', 'JPN', 'KEN', 'KHM', 'KIR', 'KNA',
       'KOR', 'KWT', 'LBN', 'LBR', 'LCA', 'LKA', 'LTU', 'LVA', 'MAF',
       'MAR', 'MCO', 'MDG', 'MDV', 'MEX', 'MHL', 'MLT', 'MMR', 'MNP',
       'MOZ', 'MRT', 'MSR', 'MTQ', 'MUS', 'MYS', 'MYT', 'NAM', 'NCL',
       'NFK', 'NGA', 'NIC', 'NIU', 'NLD', 'NOR', 'NZL', 'OMN', 'PAK',
       'PAN', 'PCN',

In [54]:
# Get only the values (skip index)
df_mat = mat[mat.columns.values[1:183]]
df_mat.head(5)

iso_ter1_y,ABW,AIA,ALB,ARE,ARG,ASM,ATA,ATF,ATG,AUS,...,USA,VCT,VEN,VGB,VIR,VNM,VUT,WSM,YEM,ZAF
0,1485,1327,61,133,187,257,3,329,1328,400,...,1435,1380,1485,1280,1329,300,325,279,116,441
1,1327,1519,58,118,204,300,4,339,1503,425,...,1462,1374,1492,1441,1498,273,352,319,111,460
2,61,58,347,31,38,24,0,49,58,48,...,78,59,61,54,57,36,28,27,23,99
3,133,118,31,1142,50,327,0,905,121,843,...,485,132,133,103,123,823,522,435,1023,816
4,187,204,38,50,671,136,32,308,203,396,...,393,217,248,182,205,93,175,162,33,386


In [55]:
# set index using countries
df_mat = df_mat.set_index(mat['index'].values) 
df_mat.head(5)

iso_ter1_y,ABW,AIA,ALB,ARE,ARG,ASM,ATA,ATF,ATG,AUS,...,USA,VCT,VEN,VGB,VIR,VNM,VUT,WSM,YEM,ZAF
ABW,1485,1327,61,133,187,257,3,329,1328,400,...,1435,1380,1485,1280,1329,300,325,279,116,441
AIA,1327,1519,58,118,204,300,4,339,1503,425,...,1462,1374,1492,1441,1498,273,352,319,111,460
ALB,61,58,347,31,38,24,0,49,58,48,...,78,59,61,54,57,36,28,27,23,99
ARE,133,118,31,1142,50,327,0,905,121,843,...,485,132,133,103,123,823,522,435,1023,816
ARG,187,204,38,50,671,136,32,308,203,396,...,393,217,248,182,205,93,175,162,33,386


In [56]:
df_mat.columns = mat['index'].values
df_mat.head(5)

,ABW,AIA,ALB,ARE,ARG,ASM,ATA,ATF,ATG,AUS,...,USA,VCT,VEN,VGB,VIR,VNM,VUT,WSM,YEM,ZAF
ABW,1485,1327,61,133,187,257,3,329,1328,400,...,1435,1380,1485,1280,1329,300,325,279,116,441
AIA,1327,1519,58,118,204,300,4,339,1503,425,...,1462,1374,1492,1441,1498,273,352,319,111,460
ALB,61,58,347,31,38,24,0,49,58,48,...,78,59,61,54,57,36,28,27,23,99
ARE,133,118,31,1142,50,327,0,905,121,843,...,485,132,133,103,123,823,522,435,1023,816
ARG,187,204,38,50,671,136,32,308,203,396,...,393,217,248,182,205,93,175,162,33,386


In [57]:
# Now it has the same shape
df_mat.shape

(181, 181)

In [58]:
# Create stewardship dictionary: for each country identify the 10 countries that share more species with it
df_sort = df_mat.copy()
steward_dict = dict.fromkeys(df_sort.columns.values)
for key in steward_dict:
    df_sort = df_sort.sort_values(by = [key], ascending=False)
    sub = df_sort[key][1:11] # to skip the same country
    vals = sub.index.values.tolist()
    vals.append(key)
    
    steward_dict[key] = json.dumps(vals)

In [59]:
# Convert stewardship dictionary into dataframe
steward_df = pd.DataFrame(steward_dict.items(), columns = ["GID_0","filter_steward"])
steward_df.head(5)

,GID_0,filter_steward
0,ABW,"[""VEN"", ""COL"", ""CUW"", ""DOM"", ""BES"", ""USA"", ""BH..."
1,AIA,"[""DOM"", ""BLM"", ""GLP"", ""PRI"", ""ATG"", ""BES"", ""MA..."
2,ALB,"[""GRC"", ""ITA"", ""ESP"", ""HRV"", ""DZA"", ""FRA"", ""TU..."
3,ARE,"[""IRN"", ""SAU"", ""OMN"", ""PAK"", ""SYC"", ""YEM"", ""MD..."
4,ARG,"[""CHL"", ""BRA"", ""NZL"", ""AUS"", ""USA"", ""ZAF"", ""NF..."


In [60]:
steward_df.shape

(181, 2)

### Get nearest EEZ
Similar to what was done in the Terrestrial_NRC notebook for the countries' centroids: the idea here is to find, for each of the displayed EEZ, the closest ones using the distance between their centroids. 

Note that the original EEZ layer often has several EEZ associated with the same GID_0 (being GID_0 the iso_ter1 or, when this is missing, the iso_sov1). This leads to problems when trying to match the EEZ and the country polygons currently displayed in the NRC. For this reason, using ArcGIS Pro, we first dissolved the EEZ layer according to the GID_0 field, so that all polygons with same GID_0 code were considered as 1. Then, the centroids were calculated using the "Feature to Point tool". The centroids were not forced to fall within the feature since most are made of several polygons and forcing the centroid to fall just in one of them can lead to weird situation such as the centroid of the Spanish EEZ falling in the Canary Islands. For this reason, the "inside" option of the tool was disabled and the resulting centroids are located at the center of all the polygons conforming each of those EEZ, even if thar means that they fall, somethimes, outside the corresponding EEZ (in the ocean or even in a different EEZ)

In [61]:
from math import radians
import pandas as pd
import numpy as np
from sklearn.metrics import DistanceMetric

In [62]:
eez2.head()

,OID_,GID_0,AREA_KM2,ORIG_FID,Latitude,Longitude
0,1,ABW,2.997030e+04,1,13.741380,-69.673412
2,3,AIA,9.015796e+04,3,20.018030,-62.543285
3,4,ALB,1.216555e+04,4,40.927084,19.113001
4,5,ARE,5.783815e+04,5,25.058911,54.030580
5,6,ARG,1.072577e+06,6,-47.089988,-62.159504


In [63]:
len(eez2)

181

In [64]:
eez2= eez2.drop(columns={'OID_','ORIG_FID'}).rename(columns={'AREA_KM2':'Area_EEZ'})

In [66]:
df_coord = pd.DataFrame(data = eez2['GID_0'])
len(df_coord)

181

In [67]:
df_coord.head()

,GID_0
0,ABW
2,AIA
3,ALB
4,ARE
5,ARG


In [68]:
len(df['GID_0'].values)

252

In [70]:
# Get the coordinates of each EEZ centroid
y_list = []
x_list = []
for index, i in eez2.iterrows():
    y_list.append(i['Latitude'])
    x_list.append(i['Longitude'])
    
df_coord['x'] = x_list
df_coord['y'] = y_list
df_coord.head(5)

,GID_0,x,y
0,ABW,-69.673412,13.741380
2,AIA,-62.543285,20.018030
3,ALB,19.113001,40.927084
4,ARE,54.030580,25.058911
5,ARG,-62.159504,-47.089988


In [71]:
# Convert the coordinates to radians
df_coord['lat'] = np.radians(df_coord['y'])
df_coord['lon'] = np.radians(df_coord['x'])
df_coord.head(5)

,GID_0,x,y,lat,lon
0,ABW,-69.673412,13.741380,0.239832,-1.216030
2,AIA,-62.543285,20.018030,0.349381,-1.091586
3,ALB,19.113001,40.927084,0.714312,0.333585
4,ARE,54.030580,25.058911,0.437361,0.943012
5,ARG,-62.159504,-47.089988,-0.821875,-1.084888


In [72]:
# Get distance metric and use it to calculate the distance between coordinates of each country
dist = DistanceMetric.get_metric('haversine')
dist_df = pd.DataFrame(dist.pairwise(df_coord[['lat','lon']].to_numpy())*6373,  columns=df_coord.GID_0.unique(), index=df_coord.GID_0.unique())
dist_df.head(5)

,ABW,AIA,ALB,ARE,ARG,ASM,ATA,ATF,ATG,AUS,...,USA,VCT,VEN,VGB,VIR,VNM,VUT,WSM,YEM,ZAF
ABW,0.000000,1030.808723,8914.511641,12547.816812,6807.636739,11363.979416,10402.096103,14419.463293,1119.079832,16480.608889,...,5516.075937,865.554410,419.428711,921.115430,641.667561,17196.124674,13781.883918,11743.613019,12950.292114,11709.958023
AIA,1030.808723,0.000000,7885.881036,11527.545191,7464.512951,12230.890200,10872.544121,14304.121273,251.553043,17497.492407,...,5649.937921,768.373044,880.534088,208.905758,396.759468,16376.658731,14630.528192,12595.968453,11985.848882,11508.891880
ALB,8914.511641,7885.881036,0.000000,3670.470318,12645.491889,16907.327294,12690.409407,10176.126307,7809.772415,14634.491854,...,9906.514331,8290.467684,8669.939067,8022.311156,8273.171738,9138.476890,16035.353774,16738.656936,4467.695397,8856.152641
ARE,12547.816812,11527.545191,3670.470318,0.000000,13972.846658,15354.787082,11798.337743,7500.825686,11431.579298,11014.248601,...,12616.292443,11862.212594,12270.933759,11674.109276,11906.979081,5928.339267,13212.308320,14961.932367,1369.323080,7497.564528
ARG,6807.636739,7464.512951,12645.491889,13972.846658,0.000000,10117.891654,3757.815605,8684.571735,7323.187947,11165.678004,...,11219.997133,6701.539762,6671.524486,7509.462226,7162.428327,15984.145870,11318.944369,10422.740599,12883.478703,7102.284726


In [73]:
dist_df.shape

(181, 181)

In [74]:
# Sort the table for each point. 0 values correspond to the same country, so take the 1:11 and add the row names to a dictionary to have the names of the 10 closest EEZ. 
dist_df_sort = dist_df.copy()
neighbour_dict = dict.fromkeys(dist_df_sort.columns.values)
for key in neighbour_dict:
    dist_df_sort = dist_df_sort.sort_values(by = [key]) # sort countries from closest to farthest
    sub = dist_df_sort[key][1:11] # keep the 10 closest ones but skip the same country (0 distance)
    vals = sub.index.values.tolist() # take the values
    neighbour_dict[key] = json.dumps(vals) # include them in dictionary

In [75]:
# Convert neighboring dictionary into dataframe
neigh_df = pd.DataFrame(neighbour_dict.items(), columns = ["GID_0","filter_neigh"])
neigh_df.head(5)

,GID_0,filter_neigh
0,ABW,"[""CUW"", ""BES"", ""VEN"", ""PRI"", ""DOM"", ""HTI"", ""VI..."
1,AIA,"[""VGB"", ""MAF"", ""BLM"", ""SXM"", ""ATG"", ""KNA"", ""VI..."
2,ALB,"[""BIH"", ""HRV"", ""ITA"", ""GRC"", ""SVN"", ""MLT"", ""BG..."
3,ARE,"[""QAT"", ""IRN"", ""KWT"", ""IRQ"", ""OMN"", ""PAK"", ""YE..."
4,ARG,"[""FLK"", ""URY"", ""CHL"", ""SGS"", ""ATA"", ""PER"", ""BR..."


In [71]:
neigh_df[neigh_df['GID_0']=='USA'].values

array([['USA',
        '["MEX", "CAN", "BLZ", "CUB", "GTM", "HND", "CYM", "SLV", "BHS", "NIC"]']],
      dtype=object)

In [77]:
neigh_df.shape

(181, 2)

In [78]:
# See in which countries the steward and neighbour dictionaries differ
list1= list(neigh_df['GID_0'])
list2=list(steward_df['GID_0'])
list3 = list(set(list1).difference(list2))
list3

[]

In [80]:
# Merge these two dataframes together
df_dict = pd.merge(left = neigh_df, right = steward_df, left_on = "GID_0", right_on = "GID_0", how = "left")
df_dict.head(5)

,GID_0,filter_neigh,filter_steward
0,ABW,"[""CUW"", ""BES"", ""VEN"", ""PRI"", ""DOM"", ""HTI"", ""VI...","[""VEN"", ""COL"", ""CUW"", ""DOM"", ""BES"", ""USA"", ""BH..."
1,AIA,"[""VGB"", ""MAF"", ""BLM"", ""SXM"", ""ATG"", ""KNA"", ""VI...","[""DOM"", ""BLM"", ""GLP"", ""PRI"", ""ATG"", ""BES"", ""MA..."
2,ALB,"[""BIH"", ""HRV"", ""ITA"", ""GRC"", ""SVN"", ""MLT"", ""BG...","[""GRC"", ""ITA"", ""ESP"", ""HRV"", ""DZA"", ""FRA"", ""TU..."
3,ARE,"[""QAT"", ""IRN"", ""KWT"", ""IRQ"", ""OMN"", ""PAK"", ""YE...","[""IRN"", ""SAU"", ""OMN"", ""PAK"", ""SYC"", ""YEM"", ""MD..."
4,ARG,"[""FLK"", ""URY"", ""CHL"", ""SGS"", ""ATA"", ""PER"", ""BR...","[""CHL"", ""BRA"", ""NZL"", ""AUS"", ""USA"", ""ZAF"", ""NF..."


In [81]:
df_dict.shape

(181, 3)

### Get below and above countries for each field in challenges

In [82]:
df.columns

Index(['GID_0', 'NAME_0', 'jpg_url', 'GNI_PPP', 'max_amph', 'max_bird',
       'max_mamm', 'max_rept', 'max_cact', 'max_coni', 'max_all', 'sentence',
       'Average SPI', 'GlobalID', 'continent', 'has_priority', 'has_raisg',
       'Area_Country', 'protection_needed_ter', 'iso2', 'max_highlited_sp',
       'x', 'y', 'hm_ter', 'hm_no_ter', 'hm_vh_ter', 'Pop2020', 'SPI_ter',
       'prop_protected_ter', 'amphibians', 'birds', 'mammals', 'reptiles',
       'endemic_amphibians', 'endemic_birds', 'endemic_mammals',
       'endemic_reptiles', 'nspecies_ter', 'total_endemic_ter',
       'filter_similar_ter', 'Pop2020_EEZ', 'SPI_mar', 'prop_protected_mar',
       'hm_no_mar', 'hm_mar', 'hm_vh_mar', 'mammals_mar', 'fishes_mar',
       'endemic_mammals_mar', 'endemic_fishes_mar', 'nspecies_mar',
       'total_endemic_mar'],
      dtype='object')

In [83]:
eez2.head()

,GID_0,Area_EEZ,Latitude,Longitude
0,ABW,2.997030e+04,13.741380,-69.673412
2,AIA,9.015796e+04,20.018030,-62.543285
3,ALB,1.216555e+04,40.927084,19.113001
4,ARE,5.783815e+04,25.058911,54.030580
5,ARG,1.072577e+06,-47.089988,-62.159504


In [84]:
len(eez2)

181

In [92]:
df = pd.merge(left = df, right = eez2, left_on = "GID_0", right_on = "GID_0", how = "left").drop(columns = {'Latitude','Longitude'})
df.columns

Index(['GID_0', 'NAME_0', 'jpg_url', 'GNI_PPP', 'max_amph', 'max_bird',
       'max_mamm', 'max_rept', 'max_cact', 'max_coni', 'max_all', 'sentence',
       'Average SPI', 'GlobalID', 'continent', 'has_priority', 'has_raisg',
       'Area_Country', 'protection_needed_ter', 'iso2', 'max_highlited_sp',
       'x', 'y', 'hm_ter', 'hm_no_ter', 'hm_vh_ter', 'Pop2020', 'SPI_ter',
       'prop_protected_ter', 'amphibians', 'birds', 'mammals', 'reptiles',
       'endemic_amphibians', 'endemic_birds', 'endemic_mammals',
       'endemic_reptiles', 'nspecies_ter', 'total_endemic_ter',
       'filter_similar_ter', 'Pop2020_EEZ', 'SPI_mar', 'prop_protected_mar',
       'hm_no_mar', 'hm_mar', 'hm_vh_mar', 'mammals_mar', 'fishes_mar',
       'endemic_mammals_mar', 'endemic_fishes_mar', 'nspecies_mar',
       'total_endemic_mar', 'Area_EEZ'],
      dtype='object')

In [93]:
df.head()

,GID_0,NAME_0,jpg_url,GNI_PPP,max_amph,max_bird,max_mamm,max_rept,max_cact,max_coni,...,hm_no_mar,hm_mar,hm_vh_mar,mammals_mar,fishes_mar,endemic_mammals_mar,endemic_fishes_mar,nspecies_mar,total_endemic_mar,Area_EEZ
0,ABW,Aruba,https://live.staticflickr.com/1952/31416683438...,3.9,3,1,10,10,10,0,...,0.0,78.304750,21.695250,19,1466,0,0,1485,0,29970.299588
1,AFG,Afghanistan,https://p1.pxfuel.com/preview/967/12/53/afghan...,70.6,10,9,10,10,0,9,...,NaN,NaN,NaN,0,0,0,0,0,0,NaN
2,AGO,Angola,https://live.staticflickr.com/3787/13698381215...,192.2,10,10,10,10,3,5,...,NaN,NaN,NaN,0,0,0,0,0,0,NaN
3,AIA,Anguilla,https://live.staticflickr.com/8063/8194570372_...,NaN,9,7,10,10,6,0,...,0.0,94.367741,5.632259,25,1494,0,0,1519,0,90157.964205
4,ALA,Åland,https://p1.pxfuel.com/preview/294/670/561/alan...,NaN,1,1,1,1,0,1,...,NaN,NaN,NaN,0,0,0,0,0,0,NaN


In [96]:
## Fill Nan in pop with zeroes
df['Pop2020_EEZ'] = df['Pop2020_EEZ'].fillna(0) 
df.head()

,GID_0,NAME_0,jpg_url,GNI_PPP,max_amph,max_bird,max_mamm,max_rept,max_cact,max_coni,...,hm_no_mar,hm_mar,hm_vh_mar,mammals_mar,fishes_mar,endemic_mammals_mar,endemic_fishes_mar,nspecies_mar,total_endemic_mar,Area_EEZ
0,ABW,Aruba,https://live.staticflickr.com/1952/31416683438...,3.9,3,1,10,10,10,0,...,0.0,78.304750,21.695250,19,1466,0,0,1485,0,29970.299588
1,AFG,Afghanistan,https://p1.pxfuel.com/preview/967/12/53/afghan...,70.6,10,9,10,10,0,9,...,NaN,NaN,NaN,0,0,0,0,0,0,NaN
2,AGO,Angola,https://live.staticflickr.com/3787/13698381215...,192.2,10,10,10,10,3,5,...,NaN,NaN,NaN,0,0,0,0,0,0,NaN
3,AIA,Anguilla,https://live.staticflickr.com/8063/8194570372_...,NaN,9,7,10,10,6,0,...,0.0,94.367741,5.632259,25,1494,0,0,1519,0,90157.964205
4,ALA,Åland,https://p1.pxfuel.com/preview/294/670/561/alan...,NaN,1,1,1,1,0,1,...,NaN,NaN,NaN,0,0,0,0,0,0,NaN


In [98]:
fields = ['GID_0', 'NAME_0', 'Area_EEZ', 'Pop2020_EEZ', 'prop_protected_mar', 'hm_vh_mar', 'total_endemic_mar', 'nspecies_mar', 'SPI_mar', "continent"]

##### % of protection needed still pending from MOL

In [99]:
df_fields = df[fields].copy()
df_fields.head(2)

,GID_0,NAME_0,Area_EEZ,Pop2020_EEZ,prop_protected_mar,hm_vh_mar,total_endemic_mar,nspecies_mar,SPI_mar,continent
0,ABW,Aruba,29970.299588,1547.605201,0.0,21.69525,0,1485,0,North America
1,AFG,Afghanistan,NaN,0.000000,NaN,NaN,0,0,0,Asia


In [100]:
filter_fields = ['Area_EEZ','Pop2020_EEZ', 'hm_vh_mar', 'prop_protected_mar', 'total_endemic_mar','nspecies_mar', 'SPI_mar']

In [101]:
# Get dictionaries for the other fields
df_sort = df.copy()
nber_index = 5
max_index = len(df_sort.index) - 1

new_fields = []
for field in filter_fields:
    df_sort = df_sort.sort_values(by = [field]).reset_index(drop=True)
    collapse_list = []
    for index, i in df_sort.iterrows():
        country_gid = df_sort.GID_0[index]
        above_index = index - nber_index
        below_index = index + nber_index + 1
        if above_index < 0:
            below_index = nber_index * 2 
            above_index = 0
        if below_index > max_index:
            above_index = max_index - (nber_index * 2)
            below_index = max_index

        sub_pd = df_sort.GID_0[above_index:below_index]
        val_list = sub_pd.values.tolist()
        collapse_list.append(json.dumps(val_list))

        #val_list_rem = val_list.remove(country_gid)
    filter_field = f"filter_{field}"
    new_fields.append(filter_field)
    df_sort[filter_field] = collapse_list

In [102]:
df_sort.head(2)

,GID_0,NAME_0,jpg_url,GNI_PPP,max_amph,max_bird,max_mamm,max_rept,max_cact,max_coni,...,nspecies_mar,total_endemic_mar,Area_EEZ,filter_Area_EEZ,filter_Pop2020_EEZ,filter_hm_vh_mar,filter_prop_protected_mar,filter_total_endemic_mar,filter_nspecies_mar,filter_SPI_mar
0,TCD,Chad,https://live.staticflickr.com/4765/39090786674...,28.6,10,8,8,7,0,0,...,0,0,NaN,"[""SRB"", ""SSD"", ""SVK"", ""SWZ"", ""SYR"", ""TCD"", ""TG...","[""LAO"", ""KGZ"", ""LIE"", ""SWZ"", ""SYR"", ""TCD"", ""TG...","[""LAO"", ""KGZ"", ""LIE"", ""SWZ"", ""SYR"", ""TCD"", ""TG...","[""LAO"", ""KGZ"", ""LIE"", ""SWZ"", ""SYR"", ""TCD"", ""TG...","[""LAO"", ""KGZ"", ""LIE"", ""SWZ"", ""SYR"", ""TCD"", ""TG...","[""TCD"", ""WLF"", ""RWA"", ""PSE"", ""PRY"", ""NRU"", ""NP...","[""TCD"", ""IRQ"", ""WSM"", ""CYM"", ""IND"", ""VGB"", ""AB..."
1,IRQ,Iraq,https://upload.wikimedia.org/wikipedia/commons...,650.5,10,7,10,10,0,8,...,172,0,1187.499037,"[""SVN"", ""MCO"", ""SXM"", ""SGP"", ""MAF"", ""IRQ"", ""JE...","[""COD"", ""GRD"", ""MYT"", ""MTQ"", ""TON"", ""IRQ"", ""BE...","[""POL"", ""ISR"", ""BIH"", ""SGP"", ""JOR"", ""IRQ"", ""JE...","[""STP"", ""BMU"", ""GUY"", ""PYF"", ""ISR"", ""IRQ"", ""SH...","[""IND"", ""BMU"", ""GUY"", ""PYF"", ""ISR"", ""IRQ"", ""SH...","[""GEO"", ""IND"", ""SJM"", ""ATA"", ""GRL"", ""IRQ"", ""BE...","[""TCD"", ""IRQ"", ""WSM"", ""CYM"", ""IND"", ""VGB"", ""AB..."


In [103]:
len(df_sort)

252

In [104]:
df_sort = df_sort[df_sort['Area_EEZ']>0]
len(df_sort)

181

### Get countries from same continent

In [105]:
df_sort.continent.unique()

array(['Asia', 'Oceania', 'North America', 'Africa', 'Europe',
       'South America', 'Antarctica'], dtype=object)

In [106]:
continent_dict = {}

In [107]:
for continent in df_sort.continent.unique():
    countries = df_sort.loc[df_sort['continent'] == continent].GID_0
    continent_dict[continent] = countries.tolist()

In [108]:
# Create same continent dictionary
same_continent_dict = dict.fromkeys(df_sort.GID_0.unique())
for key in same_continent_dict:
    continent_name = df_sort.loc[df_sort['GID_0'] == key, "continent"].to_list()[0]
    vals = continent_dict[continent_name]
    same_continent_dict[key] = json.dumps(vals)

In [109]:
# Convert to dataframe
continent_df = pd.DataFrame(same_continent_dict.items(), columns = ["GID_0","filter_continent"])
continent_df.head()

,GID_0,filter_continent
0,IRQ,"[""IRQ"", ""IND"", ""LBN"", ""PRK"", ""KWT"", ""YEM"", ""SG..."
1,WSM,"[""WSM"", ""TKL"", ""VUT"", ""PYF"", ""FSM"", ""CCK"", ""TO..."
2,CYM,"[""CYM"", ""VGB"", ""ABW"", ""LCA"", ""GRD"", ""SPM"", ""TT..."
3,IND,"[""IRQ"", ""IND"", ""LBN"", ""PRK"", ""KWT"", ""YEM"", ""SG..."
4,VGB,"[""CYM"", ""VGB"", ""ABW"", ""LCA"", ""GRD"", ""SPM"", ""TT..."


In [110]:
continent_df.shape

(181, 2)

In [111]:
# Merge the 3 filters we have so far into one dataframe
df_dict = pd.merge(left = continent_df, 
                   right = df_dict, left_on = "GID_0", right_on = "GID_0", how = "left")

In [112]:
df_dict.shape

(181, 4)

In [113]:
df_dict.head()

,GID_0,filter_continent,filter_neigh,filter_steward
0,IRQ,"[""IRQ"", ""IND"", ""LBN"", ""PRK"", ""KWT"", ""YEM"", ""SG...","[""KWT"", ""IRN"", ""QAT"", ""ARE"", ""SAU"", ""JOR"", ""LB...","[""KWT"", ""SYC"", ""MDG"", ""MUS"", ""MOZ"", ""KEN"", ""TZ..."
1,WSM,"[""WSM"", ""TKL"", ""VUT"", ""PYF"", ""FSM"", ""CCK"", ""TO...","[""ASM"", ""TKL"", ""NIU"", ""TON"", ""COK"", ""VUT"", ""SL...","[""AUS"", ""COK"", ""KIR"", ""USA"", ""NCL"", ""FJI"", ""TO..."
2,CYM,"[""CYM"", ""VGB"", ""ABW"", ""LCA"", ""GRD"", ""SPM"", ""TT...","[""CUB"", ""HND"", ""JAM"", ""BLZ"", ""NIC"", ""HTI"", ""BH...","[""CUB"", ""HTI"", ""JAM"", ""DOM"", ""BHS"", ""MEX"", ""HN..."
3,IND,"[""IRQ"", ""IND"", ""LBN"", ""PRK"", ""KWT"", ""YEM"", ""SG...","[""LKA"", ""BGD"", ""MDV"", ""MMR"", ""PAK"", ""THA"", ""KH...","[""IND"", ""MMR"", ""THA"", ""IDN"", ""MYS"", ""LKA"", ""VN..."
4,VGB,"[""CYM"", ""VGB"", ""ABW"", ""LCA"", ""GRD"", ""SPM"", ""TT...","[""AIA"", ""MAF"", ""SXM"", ""BLM"", ""PRI"", ""VIR"", ""KN...","[""DOM"", ""PRI"", ""HTI"", ""CUB"", ""AIA"", ""VIR"", ""BL..."


In [114]:
df_dict[df_dict['filter_continent'].isnull()]

,GID_0,filter_continent,filter_neigh,filter_steward


### Create the `filter_similar_marine` field with all the filters together

In [115]:
new_fields

['filter_Area_EEZ',
 'filter_Pop2020_EEZ',
 'filter_hm_vh_mar',
 'filter_prop_protected_mar',
 'filter_total_endemic_mar',
 'filter_nspecies_mar',
 'filter_SPI_mar']

In [116]:
new_fields.append("filter_neigh")
new_fields.append("filter_steward")
new_fields.append("filter_continent")
new_fields

['filter_Area_EEZ',
 'filter_Pop2020_EEZ',
 'filter_hm_vh_mar',
 'filter_prop_protected_mar',
 'filter_total_endemic_mar',
 'filter_nspecies_mar',
 'filter_SPI_mar',
 'filter_neigh',
 'filter_steward',
 'filter_continent']

In [117]:
df_sort.shape

(181, 60)

In [118]:
df_sort.head(1)

,GID_0,NAME_0,jpg_url,GNI_PPP,max_amph,max_bird,max_mamm,max_rept,max_cact,max_coni,...,nspecies_mar,total_endemic_mar,Area_EEZ,filter_Area_EEZ,filter_Pop2020_EEZ,filter_hm_vh_mar,filter_prop_protected_mar,filter_total_endemic_mar,filter_nspecies_mar,filter_SPI_mar
1,IRQ,Iraq,https://upload.wikimedia.org/wikipedia/commons...,650.5,10,7,10,10,0,8,...,172,0,1187.499037,"[""SVN"", ""MCO"", ""SXM"", ""SGP"", ""MAF"", ""IRQ"", ""JE...","[""COD"", ""GRD"", ""MYT"", ""MTQ"", ""TON"", ""IRQ"", ""BE...","[""POL"", ""ISR"", ""BIH"", ""SGP"", ""JOR"", ""IRQ"", ""JE...","[""STP"", ""BMU"", ""GUY"", ""PYF"", ""ISR"", ""IRQ"", ""SH...","[""IND"", ""BMU"", ""GUY"", ""PYF"", ""ISR"", ""IRQ"", ""SH...","[""GEO"", ""IND"", ""SJM"", ""ATA"", ""GRL"", ""IRQ"", ""BE...","[""TCD"", ""IRQ"", ""WSM"", ""CYM"", ""IND"", ""VGB"", ""AB..."


In [119]:
df_filter = pd.merge(left = df_sort, right = df_dict, left_on = "GID_0", right_on = "GID_0", how = "left")
df_filter.head(2)

,GID_0,NAME_0,jpg_url,GNI_PPP,max_amph,max_bird,max_mamm,max_rept,max_cact,max_coni,...,filter_Area_EEZ,filter_Pop2020_EEZ,filter_hm_vh_mar,filter_prop_protected_mar,filter_total_endemic_mar,filter_nspecies_mar,filter_SPI_mar,filter_continent,filter_neigh,filter_steward
0,IRQ,Iraq,https://upload.wikimedia.org/wikipedia/commons...,650.5,10,7,10,10,0,8,...,"[""SVN"", ""MCO"", ""SXM"", ""SGP"", ""MAF"", ""IRQ"", ""JE...","[""COD"", ""GRD"", ""MYT"", ""MTQ"", ""TON"", ""IRQ"", ""BE...","[""POL"", ""ISR"", ""BIH"", ""SGP"", ""JOR"", ""IRQ"", ""JE...","[""STP"", ""BMU"", ""GUY"", ""PYF"", ""ISR"", ""IRQ"", ""SH...","[""IND"", ""BMU"", ""GUY"", ""PYF"", ""ISR"", ""IRQ"", ""SH...","[""GEO"", ""IND"", ""SJM"", ""ATA"", ""GRL"", ""IRQ"", ""BE...","[""TCD"", ""IRQ"", ""WSM"", ""CYM"", ""IND"", ""VGB"", ""AB...","[""IRQ"", ""IND"", ""LBN"", ""PRK"", ""KWT"", ""YEM"", ""SG...","[""KWT"", ""IRN"", ""QAT"", ""ARE"", ""SAU"", ""JOR"", ""LB...","[""KWT"", ""SYC"", ""MDG"", ""MUS"", ""MOZ"", ""KEN"", ""TZ..."
1,WSM,Samoa,https://upload.wikimedia.org/wikipedia/commons...,1.2,0,10,10,10,0,0,...,"[""ATG"", ""BGD"", ""PRK"", ""HTI"", ""CYM"", ""WSM"", ""DZ...","[""FSM"", ""KNA"", ""ATG"", ""SUR"", ""VIR"", ""WSM"", ""JE...","[""LBR"", ""GTM"", ""MDV"", ""ZAF"", ""GLP"", ""WSM"", ""CH...","[""CCK"", ""QAT"", ""BRN"", ""LBN"", ""DZA"", ""WSM"", ""TU...","[""CCK"", ""QAT"", ""BRN"", ""LBN"", ""DZA"", ""WSM"", ""TU...","[""BES"", ""DOM"", ""CUB"", ""BHS"", ""VEN"", ""WSM"", ""OM...","[""TCD"", ""IRQ"", ""WSM"", ""CYM"", ""IND"", ""VGB"", ""AB...","[""WSM"", ""TKL"", ""VUT"", ""PYF"", ""FSM"", ""CCK"", ""TO...","[""ASM"", ""TKL"", ""NIU"", ""TON"", ""COK"", ""VUT"", ""SL...","[""AUS"", ""COK"", ""KIR"", ""USA"", ""NCL"", ""FJI"", ""TO..."


In [120]:
df_filter.shape

(181, 63)

In [121]:
similar_list = []
for index, i in df_filter.iterrows():
    filter_dict = i[new_fields].to_dict()   
    vals = json.dumps(filter_dict).replace('NaN','"NaN"').replace('"[', '[').replace(']"', ']').replace('\\', '')
    similar_list.append(vals)
    #similar_list.append(json.loads(json.dumps(filter_dict)))    
df_filter['filter_similar_mar'] = similar_list
df_filter.head(2)

,GID_0,NAME_0,jpg_url,GNI_PPP,max_amph,max_bird,max_mamm,max_rept,max_cact,max_coni,...,filter_Pop2020_EEZ,filter_hm_vh_mar,filter_prop_protected_mar,filter_total_endemic_mar,filter_nspecies_mar,filter_SPI_mar,filter_continent,filter_neigh,filter_steward,filter_similar_mar
0,IRQ,Iraq,https://upload.wikimedia.org/wikipedia/commons...,650.5,10,7,10,10,0,8,...,"[""COD"", ""GRD"", ""MYT"", ""MTQ"", ""TON"", ""IRQ"", ""BE...","[""POL"", ""ISR"", ""BIH"", ""SGP"", ""JOR"", ""IRQ"", ""JE...","[""STP"", ""BMU"", ""GUY"", ""PYF"", ""ISR"", ""IRQ"", ""SH...","[""IND"", ""BMU"", ""GUY"", ""PYF"", ""ISR"", ""IRQ"", ""SH...","[""GEO"", ""IND"", ""SJM"", ""ATA"", ""GRL"", ""IRQ"", ""BE...","[""TCD"", ""IRQ"", ""WSM"", ""CYM"", ""IND"", ""VGB"", ""AB...","[""IRQ"", ""IND"", ""LBN"", ""PRK"", ""KWT"", ""YEM"", ""SG...","[""KWT"", ""IRN"", ""QAT"", ""ARE"", ""SAU"", ""JOR"", ""LB...","[""KWT"", ""SYC"", ""MDG"", ""MUS"", ""MOZ"", ""KEN"", ""TZ...","{""filter_Area_EEZ"": [""SVN"", ""MCO"", ""SXM"", ""SGP..."
1,WSM,Samoa,https://upload.wikimedia.org/wikipedia/commons...,1.2,0,10,10,10,0,0,...,"[""FSM"", ""KNA"", ""ATG"", ""SUR"", ""VIR"", ""WSM"", ""JE...","[""LBR"", ""GTM"", ""MDV"", ""ZAF"", ""GLP"", ""WSM"", ""CH...","[""CCK"", ""QAT"", ""BRN"", ""LBN"", ""DZA"", ""WSM"", ""TU...","[""CCK"", ""QAT"", ""BRN"", ""LBN"", ""DZA"", ""WSM"", ""TU...","[""BES"", ""DOM"", ""CUB"", ""BHS"", ""VEN"", ""WSM"", ""OM...","[""TCD"", ""IRQ"", ""WSM"", ""CYM"", ""IND"", ""VGB"", ""AB...","[""WSM"", ""TKL"", ""VUT"", ""PYF"", ""FSM"", ""CCK"", ""TO...","[""ASM"", ""TKL"", ""NIU"", ""TON"", ""COK"", ""VUT"", ""SL...","[""AUS"", ""COK"", ""KIR"", ""USA"", ""NCL"", ""FJI"", ""TO...","{""filter_Area_EEZ"": [""ATG"", ""BGD"", ""PRK"", ""HTI..."


In [122]:
df_merge = df_filter[["GID_0", "filter_similar_mar"]]

In [123]:
df_merge.shape

(181, 2)

In [124]:
df_merge.head(2)

,GID_0,filter_similar_mar
0,IRQ,"{""filter_Area_EEZ"": [""SVN"", ""MCO"", ""SXM"", ""SGP..."
1,WSM,"{""filter_Area_EEZ"": [""ATG"", ""BGD"", ""PRK"", ""HTI..."


In [125]:
df.columns

Index(['GID_0', 'NAME_0', 'jpg_url', 'GNI_PPP', 'max_amph', 'max_bird',
       'max_mamm', 'max_rept', 'max_cact', 'max_coni', 'max_all', 'sentence',
       'Average SPI', 'GlobalID', 'continent', 'has_priority', 'has_raisg',
       'Area_Country', 'protection_needed_ter', 'iso2', 'max_highlited_sp',
       'x', 'y', 'hm_ter', 'hm_no_ter', 'hm_vh_ter', 'Pop2020', 'SPI_ter',
       'prop_protected_ter', 'amphibians', 'birds', 'mammals', 'reptiles',
       'endemic_amphibians', 'endemic_birds', 'endemic_mammals',
       'endemic_reptiles', 'nspecies_ter', 'total_endemic_ter',
       'filter_similar_ter', 'Pop2020_EEZ', 'SPI_mar', 'prop_protected_mar',
       'hm_no_mar', 'hm_mar', 'hm_vh_mar', 'mammals_mar', 'fishes_mar',
       'endemic_mammals_mar', 'endemic_fishes_mar', 'nspecies_mar',
       'total_endemic_mar', 'Area_EEZ'],
      dtype='object')

In [126]:
df = pd.merge(left = df, right = df_merge, left_on = "GID_0", right_on = "GID_0", how = "left")
df.head(1)

,GID_0,NAME_0,jpg_url,GNI_PPP,max_amph,max_bird,max_mamm,max_rept,max_cact,max_coni,...,hm_mar,hm_vh_mar,mammals_mar,fishes_mar,endemic_mammals_mar,endemic_fishes_mar,nspecies_mar,total_endemic_mar,Area_EEZ,filter_similar_mar
0,ABW,Aruba,https://live.staticflickr.com/1952/31416683438...,3.9,3,1,10,10,10,0,...,78.30475,21.69525,19,1466,0,0,1485,0,29970.299588,"{""filter_Area_EEZ"": [""CUW"", ""GRD"", ""LVA"", ""DMA..."


In [127]:
df.to_csv(f'{path}/NRC_Marine_20220413.csv')